In [ ]:
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import time
from datetime import datetime

spark_session = SparkSession \
    .builder \
    .master("spark://spark-master:7077") \
    .appName("Finn Test") \
    .config("spark.dynamicAllocation.enabled", True) \
    .config("spark.dynamicAllocation.shuffleTracking.enabled",True) \
    .config("spark.shuffle.service.enabled", False) \
    .config("spark.driver.port", 9999) \
    .config("spark.blockManager.port", 10005) \
    .config("spark.dynamicAllocation.executorIdleTimeout","30s") \
    .getOrCreate()

sc = spark_session.sparkContext
sqlContext = SQLContext(spark_session.sparkContext)

In [ ]:
start_time = time.time()
df = spark_session.read.json('hdfs://hdfs:9000/user/ubuntu/corpus-webis-tldr-17.json')
df.show()